In [381]:
import time
from sympy import *

import numpy as np

In [382]:
x = symbols('x')

# Funciones

In [383]:
def biseccion(a, b, func, tol, iterMax):
    """

    Función de la falsa posición para encontrar el cero de una función.
    
    Parámetros:
        a: límite inferior
        b: límite superior
        func: Función a la que se le buscará el cero.
        tol: Tolerancia para detener el método.
        iterMax: Número máximo de iteraciones permitidas.
    
    Resultados:
        xk: Aproximación de la raíz encontrada.
        error: Valor absoluto de f(x), que se toma como el error en la aproximación.
        k: Número de iteraciones realizadas.
    """
    
    f = lambdify(x,func)
    xk=None
    error = abs(f(a+b/2)) #error de x1
    k=iterMax
    
    for i in range(iterMax):
        if f(a)*f(b) < 0: #condición del teorema de Bolzano
            xk = (a+b)/2
            error = abs(f(xk)) 
            if f(xk)*f(a) < 0: #condición del teorema de Bolzano para la siguiente iteración
                b=xk
            else:
                a=xk
            if error <= tol:
                k=i
                break
        else:
            xk=None
    
    return xk,error,k

In [384]:
def falsa_posicion(a, b, func, tol, iterMax):
    """

    Función de la falsa posición para encontrar el cero de una función.
    
    Parámetros:
        a: límite inferior
        b: límite superior
        func: Función a la que se le buscará el cero.
        tol: Tolerancia para detener el método.
        iterMax: Número máximo de iteraciones permitidas.
    
    Resultados:
        xk: Aproximación de la raíz encontrada.
        error: Valor absoluto de f(x), que se toma como el error en la aproximación.
        k: Número de iteraciones realizadas.
    """
    
    f = lambdify(x,func)
    xk=b
    error = abs(f(a+b/2))
    k=iterMax
    
    x_c = xk #incialización del x actual
    x_p = 0 #inicialización del x anterior, el valor no importa
    
    for i in range(iterMax):
        if f(a)*f(b) < 0:
            xk = xk - (f(x_c)*(x_c - x_p))/(f(x_c)-f(x_p))
            x_p=x_c #actualiza el x anterior
            x_c=xk #actualiza el x actual
            error = abs(f(xk))
            if f(xk)*f(a) < 0: #condición de teorema de Bolzano
                b=xk
            else:
                a=xk
            if error <= tol:
                k=i
                break
        else:
            xk=None
    
    return xk,error,k

In [385]:
def steffensen_helper(x0, func, iterMax):
    """

    Función Steffensen para encontrar la k-ésima iteración
    
    Parámetros:
        x0: Aproximación inicial.
        func: Función cuya raíz se desea encontrar.
        iterMax: Número máximo de iteraciones permitidas.
    
    Resultados:
        xk: aproximación de la k-ésima iteración
    """
    # Convertir la función simbólica a una función numérica
    f = lambdify(x, func)
    xk = x0
    
    # Iterar hasta alcanzar el número máximo de iteraciones
    for i in range(iterMax):
        f_xk = f(xk) # Calcular el valor de la función en la aproximación actual
        f_xk_plus_f_xk = f(xk + f_xk) # Calcular f(xk + f(xk))
         # Verificar si se está aproximando a una división por cero
        if abs(f_xk_plus_f_xk - f_xk) < 1e-10:
            break  # Salir del bucle si la división por cero es inminente
        # Calcular el cuadrado de f(xk)
        f_2 = f_xk**2
        # Actualizar la aproximación utilizando el método de Steffensen
        xk = xk - f_2 / (f_xk_plus_f_xk - f_xk)
    
    return xk


In [386]:
def steffensen_secante(x0, func, tol, iterMax):
    """
    Método de Steffensen-Secante para encontrar el cero de una función.

    Parámetros:
        x0: Aproximación inicial.
        func: Función a la que se le encontrara el cero.
        tol: Tolerancia para detener el método.
        iterMax: Número máximo de iteraciones permitidas.

    Resultados:
        xk: Aproximación de la raíz encontrada.
        error: Valor absoluto de f(xk), que se toma como el error en la aproximación.
        k: Número de iteraciones realizadas.
    """
    # Convertir la función simbólica a una función numérica
    f = lambdify(x, func)
    
     # Inicializar variables
    xk = x0
    k = 0
    
    # Iterar hasta alcanzar el número máximo de iteraciones
    for i in range(1, iterMax):
        # Calcular el valor de la función en la aproximación actual
        fx = f(xk)
        
        # Calcular funcev1, funcev2 y funcev3
        funcev1 = fx**3
        funcev2 = f(xk + fx) - fx
        steff_x = steffensen_helper(xk, func, i)
        funcev3 = f(xk) - f(steff_x)

        # Actualizar la aproximación utilizando el método de Steffensen-Secante
        xk = xk - funcev1 / (funcev2 * funcev3)

        # Calcular el nuevo error absoluto (valor absoluto de f(xk))
        error = abs(f(xk))

        # Comprobar si se ha alcanzado la tolerancia deseada
        if error < tol:
            k = i
            break # Salir del bucle si se cumple la condición de parada
        
    return xk, error, k


In [387]:
def noor_one_point(f, x0, tol, iterMax):
    """

    Función de Noor's One-Point para encontrar la raíz de una ecuación.
    
    Parámetros:
        f: Función a la que se le buscará la raíz.
        x0: Aproximación inicial.
        tol: Tolerancia para detener el método.
        iterMax: Número máximo de iteraciones permitidas.
    
    Resultados:
        x: Aproximación de la raíz encontrada.
        error: Valor absoluto de f(x), que se toma como el error en la aproximación.
        k: Número de iteraciones realizadas.
        tiempo: Tiempo de ejecución del método en segundos.
    """
    
    # Iniciar el temporizador para medir el tiempo de ejecución
    start_time = time.time()
    
    # Inicializar variables
    x = x0
    k = 0           # Contador de iteraciones
    error = float('inf')     # Error inicial (inicializado en infinito)
    
    # Iterar hasta alcanzar el número máximo de iteraciones
    while k < iterMax:
        # Calcular el valor de la función en la aproximación actual
        f_x = f(x)
        
        # Calcular la derivada numérica en la aproximación actual utilizando diferencias finitas
        f_prime_x = (f(x + tol) - f(x - tol)) / (2 * tol)

        # Manejar el caso donde la derivada es aproximadamente cero
        if abs(f_prime_x) < 1e-10:
            # Evitar la división por cero, ajustando ligeramente la derivada
            f_prime_x = 1e-10
        
        # Actualizar la aproximación utilizando el método de Noor's One-Point de segundo orden
        x = x - f_x / f_prime_x
        
        # Calcular el nuevo error absoluto (valor absoluto de f(x))
        error = abs(f_x)
        
        # Comprobar si se ha alcanzado la tolerancia deseada
        if error < tol:
            break  # Salir del bucle si se cumple la condición de parada
        
        # Incrementar el contador de iteraciones
        k += 1
    
    # Detener el temporizador y registrar el tiempo de ejecución
    tiempo = time.time() - start_time
    
    return x, error, k, tiempo
    

In [388]:
def f(x):
    return exp(x)-x-10

# Testing

In [389]:
start = time.time()
xk, err, k = biseccion(-2,3, exp(x)-x-10,1e-10, 1000)
stop = time.time()

print("Xk:", xk)
print("Error:", err)
print("Número de iteraciones:", k)
print(f"Tiempo de ejecución: {stop - start}", "segundos")

Xk: 2.527963201981038
Error: 1.3100631690576847e-11
Número de iteraciones: 30
Tiempo de ejecución: 0.0009992122650146484 segundos


In [390]:
start = time.time()
xk, err, k = falsa_posicion(-2,3, exp(x)-x-10,1e-10, 1000)
stop = time.time()

print("Xk:", xk)
print("Error:", err)
print("Número de iteraciones:", k)
print(f"Tiempo de ejecución: {stop - start}", "segundos")

Xk: 2.527963201982135
Error: 4.511946372076636e-13
Número de iteraciones: 7
Tiempo de ejecución: 0.0010001659393310547 segundos


In [391]:
start = time.time()
xk= steffensen_helper(3, exp(x)-x-10, 3)
stop = time.time()

print("Xk:", xk)
print(f"Tiempo de ejecución: {stop - start}", "segundos")

Xk: 2.99351528483724
Tiempo de ejecución: 0.0009992122650146484 segundos


In [392]:
start = time.time()
xk, err, k = steffensen_secante(3, exp(x)-x-10,1e-10, 1000)
stop = time.time()

print("Xk:", xk)
print("Error:", err)
print("Número de iteraciones:", k)
print(f"Tiempo de ejecución: {stop - start}", "segundos")

Xk: 2.5279632019821743
Error: 0.0
Número de iteraciones: 7
Tiempo de ejecución: 0.00500178337097168 segundos


In [393]:
# Llama a la función Noor_One_point
x0 = 3.0
tol = 1e-10
iterMax = 1000

start = time.time()
xk, err, k, tiempo_ejec = noor_one_point(f, x0, tol, iterMax)
stop = time.time()

print("Xk:", xk)
print("Error:", err)
print("Número de iteraciones:", k)
print("Tiempo de ejecución:", "{:.10f}".format(tiempo_ejec), "segundos")

Xk: 2.52796320198217
Error: 0
Número de iteraciones: 5
Tiempo de ejecución: 0.0009999275 segundos
